In [ ]:
import json
import os
import requests

In [ ]:
HOST = "http://127.0.0.1"
PORT = "3090"

WEAVIATE_ENDPOINT = f"{HOST}:{PORT}/api/weaviate"
BASE_PATH = "data/haarlem"

In [ ]:
def create_transcript_collection():
    requests.post(
        f"{WEAVIATE_ENDPOINT}/createCollection",
        json={
            "name": "transcripts",
            "vector_index_hnsw": True,
            "distance_config": "cosine",
            "properties": [
                {"name": "text", "data_type": "text"},
                {"name": "bmContext", "data_type": "text"},
                {"name": "start", "data_type": "number"},
                {"name": "end", "data_type": "number"},
                {"name": "code", "data_type": "text"},
                {"name": "year", "data_type": "text"},
                {"name": "government", "data_type": "text"},
            ],
        },
    )


def create_diorization_collection():
    requests.post(
        f"{WEAVIATE_ENDPOINT}/createCollection",
        json={
            "name": "diorizations",
            "vector_index_hnsw": True,
            "distance_config": "cosine",
            "properties": [
                {"name": "startEnds", "data_type": "objectList"},
                {"name": "code", "data_type": "text"},
                {"name": "year", "data_type": "text"},
                {"name": "government", "data_type": "text"},
            ],
        },
    )


# create_transcript_collection()
create_diorization_collection()

In [ ]:
def add_transcripts():
    for year in os.listdir(BASE_PATH):
        if not os.path.isdir(f"{BASE_PATH}/{year}") or not os.path.isdir(
            f"{BASE_PATH}/{year}/objects"
        ):
            continue
        print("Doing", year)
        for meeting in os.listdir(f"{BASE_PATH}/{year}/objects"):
            if not meeting.endswith(".json"):
                continue
            with open(f"{BASE_PATH}/{year}/objects/{meeting}", "r") as f:
                objects = json.load(f)
                requests.post(
                    f"{WEAVIATE_ENDPOINT}/insert",
                    json={
                        "collection": "transcripts",
                        "objects": objects,
                    },
                )

add_transcripts()

In [ ]:
def add_diorizations():
    for year in os.listdir(BASE_PATH):
        if not os.path.isdir(f"{BASE_PATH}/year") or not os.path.isdir(
            f"{BASE_PATH}/year/DiorizationObjects"
        ):
            continue
        for meeting in os.listdir(f"{BASE_PATH}/year/DiorizationObjects"):
            if not meeting.endswith(".json"):
                continue
            with open(f"{BASE_PATH}/year/DiorizationObjects/{meeting}", "r") as f:
                objects = json.load(f)
                requests.post(
                    f"{WEAVIATE_ENDPOINT}/insert",
                    json={
                        "collection": "diorizations",
                        "objects": objects,
                    },
                )